In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
data = pd.read_csv('C:\\Users\\LENOVO\\Desktop\\PythonClass\\Data Science Assignements\\Assignement-12\\SalaryData_Test.csv')
data.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [19]:
data.info

<bound method DataFrame.info of        age      workclass      education  educationno        maritalstatus  \
0       25        Private           11th            7        Never-married   
1       38        Private        HS-grad            9   Married-civ-spouse   
2       28      Local-gov     Assoc-acdm           12   Married-civ-spouse   
3       44        Private   Some-college           10   Married-civ-spouse   
4       34        Private           10th            6        Never-married   
...    ...            ...            ...          ...                  ...   
15055   33        Private      Bachelors           13        Never-married   
15056   39        Private      Bachelors           13             Divorced   
15057   38        Private      Bachelors           13   Married-civ-spouse   
15058   44        Private      Bachelors           13             Divorced   
15059   35   Self-emp-inc      Bachelors           13   Married-civ-spouse   

               occupation    re

In [20]:
data.isnull().sum()

age              0
workclass        0
education        0
educationno      0
maritalstatus    0
occupation       0
relationship     0
race             0
sex              0
capitalgain      0
capitalloss      0
hoursperweek     0
native           0
Salary           0
dtype: int64

In [21]:
import re 
import string

def clean_maritalstatus(maritalstatus):
   
    maritalstatus = maritalstatus.lower()
    maritalstatus = re.sub('\[.*?\]', '', maritalstatus)
    maritalstatus = re.sub('[%s]' % re.escape(string.punctuation), '', maritalstatus)
    maritalstatus = re.sub('\w*\d\w*', '', maritalstatus)
    maritalstatus = re.sub("[0-9" "]+"," ",maritalstatus)
    maritalstatus = re.sub('[‘’“”…]', '', maritalstatus)
    return maritalstatus

clean = lambda x: clean_maritalstatus(x)

In [22]:
data['maritalstatus'] = data.maritalstatus.apply(clean)
data.maritalstatus

0             nevermarried
1         marriedcivspouse
2         marriedcivspouse
3         marriedcivspouse
4             nevermarried
               ...        
15055         nevermarried
15056             divorced
15057     marriedcivspouse
15058             divorced
15059     marriedcivspouse
Name: maritalstatus, Length: 15060, dtype: object

In [23]:
freq = pd.Series(' '.join(data['maritalstatus']).split()).value_counts()[:20] 
freq

marriedcivspouse       6990
nevermarried           4872
divorced               2083
separated               472
widowed                 450
marriedspouseabsent     182
marriedafspouse          11
dtype: int64

In [24]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['maritalstatus'] = data['maritalstatus'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [25]:
freq_Sw = pd.Series(' '.join(data['maritalstatus']).split()).value_counts()[:20] # for top 20
freq_Sw

marriedcivspouse       6990
nevermarried           4872
divorced               2083
separated               472
widowed                 450
marriedspouseabsent     182
marriedafspouse          11
dtype: int64

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
vectorizer = CountVectorizer(min_df = 1, max_df = 0.9)
X = vectorizer.fit_transform(data["maritalstatus"])
word_freq_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'occurrences':np.asarray(X.sum(axis=0)).ravel().tolist()})
word_freq_df['frequency'] = word_freq_df['occurrences']/np.sum(word_freq_df['occurrences'])

In [46]:
word_freq_df.head(30)

,term,occurrences,frequency
0,divorced,2083,0.138313
1,marriedafspouse,11,0.000730
2,marriedcivspouse,6990,0.464143
3,marriedspouseabsent,182,0.012085
4,nevermarried,4872,0.323506
5,separated,472,0.031341
6,widowed,450,0.029880


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, max_df = 0.5, smooth_idf=True) 
doc_vec = vectorizer.fit_transform(data["maritalstatus"])
names_features = vectorizer.get_feature_names()
dense = doc_vec.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns = names_features)

In [48]:
df

,divorced,marriedafspouse,marriedcivspouse,marriedspouseabsent,nevermarried,separated,widowed
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
15055,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15056,1.0,0.0,0.0,0.0,0.0,0.0,0.0
15057,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15058,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2,2), 
            max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]

In [59]:
top2_words = get_top_n2_words(data["maritalstatus"], n= 2000)
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
top2_df.head()

ValueError: empty vocabulary; perhaps the documents only contain stop words